In [1]:
import pandas as pd
import numpy as np
import itertools
import igraph
import sys
import collections
import datetime
import random
import itertools

In [22]:
period = 'EM'
trial = 2
#root = 'D:/RST/notebook/'
root = '/Volumes/Data2/RST/notebook/'

# Creating a Key for Node IDs and Node Names

In [23]:
node_data = pd.read_csv(period + '/' + str(trial) + '/od_travel_times-' + period + '-' + str(trial) + '.csv')

In [24]:
#node_90 = node_data[node_data['cutoff_time'] < 90].copy()

In [25]:
G = igraph.Graph.Read_GraphML(f = root + 'networks/' + period + '-TE-16-5-constrained.graphml')

In [26]:
node_key_table = []

for i in G.vs:
    node_key_table.append([i.index,  i['id']])
node_df_new = pd.DataFrame.from_records(node_key_table, columns = ['index', 'node_id'])

del node_key_table
node_df_new

,index,node_id
0,0,13464667-0-4-0
1,1,13464667-0-4-1
2,2,13464667-0-4-2
3,3,13464667-0-4-3
4,4,13464667-0-4-4
...,...,...
1862637,1862637,1081-9999-9-26
1862638,1862638,1081-9999-9-27
1862639,1862639,1081-9999-9-28
1862640,1862640,1081-9999-9-29


In [27]:
node_df_new['INT_ID'] = node_df_new['node_id'].str.split('-', expand = True)[0]
node_df_new['route'] = node_df_new['node_id'].str.split('-', expand = True)[1]

In [28]:
node_df_new['route_int'] = node_df_new['INT_ID'] + '-' + node_df_new['route']
node_df_new

,index,node_id,INT_ID,route,route_int
0,0,13464667-0-4-0,13464667,0,13464667-0
1,1,13464667-0-4-1,13464667,0,13464667-0
2,2,13464667-0-4-2,13464667,0,13464667-0
3,3,13464667-0-4-3,13464667,0,13464667-0
4,4,13464667-0-4-4,13464667,0,13464667-0
...,...,...,...,...,...
1862637,1862637,1081-9999-9-26,1081,9999,1081-9999
1862638,1862638,1081-9999-9-27,1081,9999,1081-9999
1862639,1862639,1081-9999-9-28,1081,9999,1081-9999
1862640,1862640,1081-9999-9-29,1081,9999,1081-9999


In [29]:
node_dict_new = node_df_new[['INT_ID']].to_dict()['INT_ID']
route_dict_new = node_df_new[['INT_ID', 'route']].to_dict()['route']

In [32]:
node_data[(node_data['origin'] == 1562)]

,origin,destination,o_node,d_node,base_time,buffer_length,cutoff_time,arrival_time,arrival_hour,arrival_minute,constrained_node
151,1562,1078,1562-0-4-23,1078-0-99-99,195.0,10.0,205.0,228,7,48,1078-9999-7-48
152,1562,1607,1562-0-4-23,1607-0-99-99,111.0,9.5,120.5,143,6,23,1607-9999-6-23


In [31]:
for row_no in range(2,5, 1):

    origin = node_data.loc[row_no,'o_node']
    dest = node_data.loc[row_no,'constrained_node']

    o = G.vs.select(id_eq = origin)[0]
    d = G.vs.select(id_eq = dest)[0]

    o_str = node_data.loc[row_no,'origin']
    d_str = node_data.loc[row_no,'destination']

    out = G.get_all_shortest_paths(o, d, weights = 'cost',  mode = 'out')

    print(len(out), origin, dest)
    
    table = []
    #full_table = []
    #route_int_table = []
    route_table = []
    for path in out:
        temp_list = list(dict.fromkeys(list(map(node_dict_new.get, path))))
        route_list = list(dict.fromkeys(list(map(route_dict_new.get, path))))
        #route_int_list = list(dict.fromkeys(list(map(route_int_dict_new.get, path))))
        #full_list = list(set(map(full_dict_new.get, path)))
        table.append(temp_list)
        route_table.append(route_list)
        #route_int_table.append(route_int_list)
        #full_table.append(full_list)

    table.sort()
    table_sort = list(x for x,_ in itertools.groupby(table))

    route_table.sort()
    route_table_sort = list(x for x,_ in itertools.groupby(route_table))

    route_df = pd.DataFrame(pd.Series(route_table_sort), columns = ['route']).reset_index().rename(
        columns = {'index':'path'}).explode('route')
    route_df['route'] = route_df['route'].astype(int)
    route_df = route_df[~route_df['route'].isin([0,9999])]
    route_df.to_csv(period + '/' + str(trial) + '/route-path_'+ str(o_str) + '_' + str(d_str) + '.csv', index = False)

    table_df = pd.DataFrame(pd.Series(table_sort), columns = ['INT_ID']).reset_index().rename(
        columns = {'index':'path'}).explode('INT_ID')
    table_df.to_csv(period + '/' + str(trial) + '/int-path_'+ str(o_str) + '_' + str(d_str) + '.csv', index = False)
    
    del table_df
    del route_df
    del table
    del route_table
    del out

32659 1021-0-4-23 1386-9999-6-52
20 1021-0-4-23 1519-9999-5-22
18 1021-0-4-23 1545-9999-5-30


## Single OD

In [33]:
row_no = 152

origin = node_data.loc[row_no,'o_node']
dest = node_data.loc[row_no,'constrained_node']

o = G.vs.select(id_eq = origin)[0]
d = G.vs.select(id_eq = dest)[0]

o_str = node_data.loc[row_no,'origin']
d_str = node_data.loc[row_no,'destination']

In [34]:
o, o_str, origin

(igraph.Vertex(<igraph.Graph object at 0x7fa2906a4e50>, 1446887, {'id': '1562-0-4-23'}),
 1562,
 '1562-0-4-23')

In [35]:
d, d_str, dest

(igraph.Vertex(<igraph.Graph object at 0x7fa2906a4e50>, 1855974, {'id': '1607-9999-6-23'}),
 1607,
 '1607-9999-6-23')

In [36]:
d = G.vs.select(id_eq = '1210-9999-11-45')[0]
d

IndexError: vertex index out of range

In [37]:
out = G.get_all_shortest_paths(o, d, weights = 'cost',  mode = 'out')

In [38]:
len(out)

14688

In [39]:
node_test = node_df_new[['node_id']].to_dict()['node_id']

In [40]:
table = []
full_table = []
#route_int_table = []
route_table = []
for path in out:
    temp_list = list(dict.fromkeys(list(map(node_dict_new.get, path))))
    route_list = list(dict.fromkeys(list(map(route_dict_new.get, path))))
    #route_int_list = list(dict.fromkeys(list(map(route_int_dict_new.get, path))))
    full_list = list(set(map(node_test.get, path)))
    table.append(temp_list)
    route_table.append(route_list)
    #route_int_table.append(route_int_list)
    full_table.append(full_list)

In [41]:
route_table.sort()
route_table_sort = list(x for x,_ in itertools.groupby(route_table))

In [42]:
table.sort()
table_sort = list(x for x,_ in itertools.groupby(table))

In [43]:
full_table

[['13444720-0-4-59',
  '13442174-0-5-44',
  '13442174-0-5-55',
  '13442174-0-5-26',
  '13442174-0-5-41',
  '13442251-39-6-1',
  '13443509-334-5-8',
  '13444720-0-4-57',
  '13442174-39-6-0',
  '13442502-0-6-7',
  '13442417-39-6-3',
  '13442502-0-6-6',
  '13442174-0-5-46',
  '13442276-334-5-11',
  '13442174-0-5-17',
  '13442174-0-5-19',
  '13444720-0-4-49',
  '13442174-0-5-25',
  '13442074-334-5-11',
  '13442174-0-5-18',
  '1607-9999-6-23',
  '13444339-334-5-6',
  '13442174-0-5-37',
  '13444720-0-4-58',
  '13442126-334-5-12',
  '13442174-0-5-50',
  '13442174-0-5-35',
  '13442174-0-5-40',
  '13442174-0-5-23',
  '13442174-0-5-31',
  '13444720-0-4-47',
  '1562-0-4-23',
  '13442174-0-5-52',
  '13442174-0-5-56',
  '13442915-334-5-9',
  '13442336-130-6-12',
  '13442192-130-6-12',
  '13444720-0-5-3',
  '13442174-0-5-22',
  '13442174-0-5-30',
  '13444720-0-4-53',
  '13442174-0-5-24',
  '13444470-334-5-6',
  '13442174-0-5-49',
  '13442174-0-5-28',
  '13442174-0-5-34',
  '13442174-0-5-54',
  '1344

In [44]:
table

[['1562',
  '13444470',
  '13444339',
  '13444143',
  '13443860',
  '13443509',
  '13443108',
  '13442915',
  '13442715',
  '13442522',
  '13442276',
  '13442204',
  '13442218',
  '13442126',
  '13442174',
  '13442251',
  '13442417',
  '13442502',
  '13442585',
  '13442643',
  '1607'],
 ['1562',
  '13444470',
  '13444339',
  '13444143',
  '13443860',
  '13443509',
  '13443108',
  '13442915',
  '13442715',
  '13442522',
  '13442276',
  '13442204',
  '13442218',
  '13442126',
  '13442174',
  '13442251',
  '13442417',
  '13442502',
  '13442585',
  '13442643',
  '1607'],
 ['1562',
  '13444470',
  '13444339',
  '13444143',
  '13443860',
  '13443509',
  '13443108',
  '13442915',
  '13442715',
  '13442522',
  '13442276',
  '13442204',
  '13442218',
  '13442126',
  '13442174',
  '13442251',
  '13442417',
  '13442502',
  '13442585',
  '13442643',
  '1607'],
 ['1562',
  '13444470',
  '13444339',
  '13444143',
  '13443860',
  '13443509',
  '13443108',
  '13442915',
  '13442715',
  '13442522',
  '

In [45]:
len(table_sort), len(route_table_sort)

(72, 15)

In [46]:
route_df = pd.DataFrame(pd.Series(route_table_sort), columns = ['route']).reset_index().rename(
    columns = {'index':'path'}).explode('route')
route_df['route'] = route_df['route'].astype(int)
route_df = route_df[~route_df['route'].isin([0,9999])]
route_df.to_csv(period + '/' + str(trial) + '/route-path_'+ str(o_str) + '_' + str(d_str) + '.csv', index = False)
route_df

,path,route
0,0,334
0,0,134
0,0,39
1,1,334
1,1,339
1,1,134
1,1,39
2,2,334
2,2,339
2,2,353


In [47]:
table_df = pd.DataFrame(pd.Series(table_sort), columns = ['INT_ID']).reset_index().rename(
    columns = {'index':'path'}).explode('INT_ID')
table_df.to_csv(period + '/' + str(trial) + '/int-path_'+ str(o_str) + '_' + str(d_str) + '.csv', index = False)

In [48]:
table_df

,path,INT_ID
0,0,1562
0,0,13444470
0,0,13444339
0,0,13444143
0,0,13443860
...,...,...
71,71,13442174
71,71,13442251
71,71,13442417
71,71,13442502


In [49]:
del table_df
del route_df
del table
del route_table
del out